# Ноутбук со стандартными наборами параметров для оптимизации с помощью optuna

In [4]:
from functools import partial

import optuna

from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [5]:
metric = partial(f1_score, average='macro')

In [7]:
x, y = make_classification(n_samples=300, n_features=7, n_informative=4, n_classes=3)

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

# LightGBM

In [10]:
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping

In [14]:
model = LGBMClassifier(verbose=-100)
model.fit(x_train, y_train)

LGBMClassifier(verbose=-100)

In [15]:
metric(y_val, model.predict(x_val))

0.7863621317444555

In [16]:
def objective(trial):
    params = {
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 16),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'verbose': -100
    }
    
    model = LGBMClassifier(**params)
    model.fit(x_train, y_train, eval_set=(x_val, y_val), callbacks=[early_stopping(stopping_rounds=30)])
    
    preds = model.predict(x_val)
    
    return metric(y_val, preds)

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-09-04 09:14:30,848] A new study created in memory with name: no-name-6212068f-919b-441d-8bc9-05f2557a6b29
[I 2024-09-04 09:14:31,033] Trial 0 finished with value: 0.7966070202621419 and parameters: {'lambda_l1': 0.010629158465864163, 'lambda_l2': 2.130156276803011e-06, 'num_leaves': 106, 'max_depth': 4, 'n_estimators': 252, 'learning_rate': 0.016290992935986334, 'feature_fraction': 0.47363608985210615, 'bagging_fraction': 0.9719450349856082, 'bagging_freq': 2, 'min_child_samples': 16, 'class_weight': None}. Best is trial 0 with value: 0.7966070202621419.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[251]	valid_0's multi_logloss: 0.53276
Training until validation scores don't improve for 30 rounds


[I 2024-09-04 09:14:31,099] Trial 1 finished with value: 0.8094124802797894 and parameters: {'lambda_l1': 0.4425748172313832, 'lambda_l2': 1.6472745645217657e-06, 'num_leaves': 123, 'max_depth': 8, 'n_estimators': 1297, 'learning_rate': 0.046451820838650064, 'feature_fraction': 0.5011118614717749, 'bagging_fraction': 0.6074057381355334, 'bagging_freq': 6, 'min_child_samples': 45, 'class_weight': None}. Best is trial 1 with value: 0.8094124802797894.
[I 2024-09-04 09:14:31,156] Trial 2 finished with value: 0.7761392208379241 and parameters: {'lambda_l1': 0.0003676330883213585, 'lambda_l2': 9.08706831955392e-08, 'num_leaves': 183, 'max_depth': 7, 'n_estimators': 1332, 'learning_rate': 0.09858900357600621, 'feature_fraction': 0.5291877205978096, 'bagging_fraction': 0.7702043947758143, 'bagging_freq': 4, 'min_child_samples': 27, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.8094124802797894.
[I 2024-09-04 09:14:31,267] Trial 3 finished with value: 0.8166603511431099 and parame

Early stopping, best iteration is:
[142]	valid_0's multi_logloss: 0.567912
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[60]	valid_0's multi_logloss: 0.518496
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[152]	valid_0's multi_logloss: 0.52404
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 1.09861


[I 2024-09-04 09:14:31,340] Trial 5 finished with value: 0.8106581814012155 and parameters: {'lambda_l1': 0.14564224022534603, 'lambda_l2': 0.06315846063739922, 'num_leaves': 245, 'max_depth': 8, 'n_estimators': 888, 'learning_rate': 0.06195177710254759, 'feature_fraction': 0.8107416177782591, 'bagging_fraction': 0.9922806159875843, 'bagging_freq': 1, 'min_child_samples': 78, 'class_weight': None}. Best is trial 3 with value: 0.8166603511431099.
[I 2024-09-04 09:14:31,361] Trial 6 finished with value: 0.14492753623188406 and parameters: {'lambda_l1': 0.01109694678787842, 'lambda_l2': 1.7242860636548276, 'num_leaves': 79, 'max_depth': 4, 'n_estimators': 750, 'learning_rate': 0.08660529911730644, 'feature_fraction': 0.8903081204723464, 'bagging_fraction': 0.508420470140318, 'bagging_freq': 7, 'min_child_samples': 63, 'class_weight': None}. Best is trial 3 with value: 0.8166603511431099.
[I 2024-09-04 09:14:31,403] Trial 7 finished with value: 0.8093317071645245 and parameters: {'lambda_l

...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 1.09861
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[22]	valid_0's multi_logloss: 0.527628


[I 2024-09-04 09:14:37,610] Trial 48 finished with value: 0.8291204758050855 and parameters: {'lambda_l1': 2.139674747149346, 'lambda_l2': 7.918061064060112e-08, 'num_leaves': 178, 'max_depth': 8, 'n_estimators': 713, 'learning_rate': 0.15807251228944805, 'feature_fraction': 0.9071836373839529, 'bagging_fraction': 0.7665269057025205, 'bagging_freq': 2, 'min_child_samples': 13, 'class_weight': 'balanced'}. Best is trial 16 with value: 0.8633484162895928.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[46]	valid_0's multi_logloss: 0.509358
Training until validation scores don't improve for 30 rounds


[I 2024-09-04 09:14:37,777] Trial 49 finished with value: 0.8208754208754209 and parameters: {'lambda_l1': 0.15613509424685365, 'lambda_l2': 0.020000841420821775, 'num_leaves': 144, 'max_depth': 11, 'n_estimators': 337, 'learning_rate': 0.17403026516711984, 'feature_fraction': 0.9564667496034305, 'bagging_fraction': 0.72283499903246, 'bagging_freq': 3, 'min_child_samples': 9, 'class_weight': None}. Best is trial 16 with value: 0.8633484162895928.


Early stopping, best iteration is:
[16]	valid_0's multi_logloss: 0.531885


In [19]:
study.best_params

{'lambda_l1': 7.681816698813154e-08,
 'lambda_l2': 0.003105842906499414,
 'num_leaves': 213,
 'max_depth': 11,
 'n_estimators': 424,
 'learning_rate': 0.1243535960980597,
 'feature_fraction': 0.762436288524388,
 'bagging_fraction': 0.6774588423737005,
 'bagging_freq': 3,
 'min_child_samples': 7,
 'class_weight': 'balanced'}

In [24]:
model = LGBMClassifier(**study.best_params, verbose=-100)
model.fit(x_train, y_train, eval_set=(x_val, y_val), callbacks=[early_stopping(stopping_rounds=30)])

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[19]	valid_0's multi_logloss: 0.492962


LGBMClassifier(bagging_fraction=0.6774588423737005, bagging_freq=3,
               class_weight='balanced', feature_fraction=0.762436288524388,
               lambda_l1=7.681816698813154e-08, lambda_l2=0.003105842906499414,
               learning_rate=0.1243535960980597, max_depth=11,
               min_child_samples=7, n_estimators=424, num_leaves=213,
               verbose=-100)

In [25]:
metric(y_val, model.predict(x_val))

0.8633484162895928

# CatBoost

In [26]:
from catboost import CatBoostClassifier

In [29]:
def objective(trial):
    params = {
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'iterations': trial.suggest_int('iterations', 100, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'rsm': trial.suggest_float('rsm', 0.4, 1.0),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0., 1.0),  # for Bayessian bootstrap
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),  # for Bernoulli bootstrap
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 250),
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced', None]),
        'bootstrap_type': "Bernoulli"
    }
    
    model = CatBoostClassifier(**params)
    model.fit(x_train, y_train, eval_set=(x_val, y_val), early_stopping_rounds=30, verbose=100)
    
    preds = model.predict(x_val)
    
    return metric(y_val, preds)

In [30]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-09-04 09:22:57,152] A new study created in memory with name: no-name-f1af6121-b684-416c-8378-936d9d1e9770
[I 2024-09-04 09:22:57,406] Trial 0 finished with value: 0.8076504271097344 and parameters: {'l2_leaf_reg': 2.347067041802901, 'max_depth': 4, 'iterations': 898, 'learning_rate': 0.1707138613343889, 'rsm': 0.5247607285223304, 'subsample': 0.26377622076109897, 'min_data_in_leaf': 160, 'auto_class_weights': 'Balanced'}. Best is trial 0 with value: 0.8076504271097344.


0:	learn: 0.9875681	test: 0.9769801	best: 0.9769801 (0)	total: 52.3ms	remaining: 46.9s
100:	learn: 0.2003772	test: 0.5133677	best: 0.5008378 (76)	total: 151ms	remaining: 1.19s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5008377634
bestIteration = 76

Shrink model to first 77 iterations.
0:	learn: 0.7090393	test: 0.7944009	best: 0.7944009 (0)	total: 12.7ms	remaining: 13.2s


[I 2024-09-04 09:22:58,676] Trial 1 finished with value: 0.8104072398190046 and parameters: {'l2_leaf_reg': 0.11499441832907728, 'max_depth': 10, 'iterations': 1041, 'learning_rate': 0.4622282992226346, 'rsm': 0.5921691446146322, 'subsample': 0.38080018149115535, 'min_data_in_leaf': 68, 'auto_class_weights': None}. Best is trial 1 with value: 0.8104072398190046.


Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5642011884
bestIteration = 14

Shrink model to first 15 iterations.
0:	learn: 1.0730787	test: 1.0820356	best: 1.0820356 (0)	total: 12.9ms	remaining: 7.33s
100:	learn: 0.2896638	test: 0.5342133	best: 0.5329564 (99)	total: 1.04s	remaining: 4.84s
200:	learn: 0.1439101	test: 0.4918741	best: 0.4913355 (195)	total: 2.17s	remaining: 3.98s
300:	learn: 0.0904373	test: 0.4866737	best: 0.4861167 (298)	total: 3.25s	remaining: 2.89s


[I 2024-09-04 09:23:02,372] Trial 2 finished with value: 0.8382703730102162 and parameters: {'l2_leaf_reg': 1.4625044862694558, 'max_depth': 8, 'iterations': 569, 'learning_rate': 0.045520048423099035, 'rsm': 0.7530665875819506, 'subsample': 0.25499045368445405, 'min_data_in_leaf': 228, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 2 with value: 0.8382703730102162.


...

100:	learn: 0.0525184	test: 0.5169299	best: 0.5124289 (74)	total: 3.6s	remaining: 27.8s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5124288525
bestIteration = 74

Shrink model to first 75 iterations.
0:	learn: 0.9836330	test: 1.0086279	best: 1.0086279 (0)	total: 12.9ms	remaining: 10s


[I 2024-09-04 09:24:20,835] Trial 47 finished with value: 0.8303888038596957 and parameters: {'l2_leaf_reg': 0.0012809483430947428, 'max_depth': 9, 'iterations': 774, 'learning_rate': 0.09168238297418571, 'rsm': 0.6955115661070053, 'subsample': 0.7116632537317986, 'min_data_in_leaf': 242, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 35 with value: 0.8510703912442845.


Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5018984995
bestIteration = 38

Shrink model to first 39 iterations.
0:	learn: 0.7774858	test: 0.9082126	best: 0.9082126 (0)	total: 11.8ms	remaining: 8.14s


[I 2024-09-04 09:24:21,452] Trial 48 finished with value: 0.7967019405312014 and parameters: {'l2_leaf_reg': 0.00015926364140405073, 'max_depth': 8, 'iterations': 693, 'learning_rate': 0.2753200348021803, 'rsm': 0.9418680034894809, 'subsample': 0.39749937361938137, 'min_data_in_leaf': 229, 'auto_class_weights': None}. Best is trial 35 with value: 0.8510703912442845.
[I 2024-09-04 09:24:21,565] Trial 49 finished with value: 0.8384241717672204 and parameters: {'l2_leaf_reg': 0.012982796782222016, 'max_depth': 2, 'iterations': 949, 'learning_rate': 0.4220551725298476, 'rsm': 0.73530559428117, 'subsample': 0.5776860696296989, 'min_data_in_leaf': 204, 'auto_class_weights': 'Balanced'}. Best is trial 35 with value: 0.8510703912442845.


Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5290441696
bestIteration = 20

Shrink model to first 21 iterations.
0:	learn: 0.8309503	test: 0.7820299	best: 0.7820299 (0)	total: 475us	remaining: 451ms
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5311589969
bestIteration = 25

Shrink model to first 26 iterations.


In [31]:
study.best_params

{'l2_leaf_reg': 1.287998582576072,
 'max_depth': 8,
 'iterations': 833,
 'learning_rate': 0.13427444108187384,
 'rsm': 0.7999348996814081,
 'subsample': 0.6590987107818918,
 'min_data_in_leaf': 205,
 'auto_class_weights': 'SqrtBalanced'}

In [33]:
model = CatBoostClassifier(**study.best_params, bootstrap_type='Bernoulli')
model.fit(x_train, y_train, eval_set=(x_val, y_val), early_stopping_rounds=30, verbose=100)

0:	learn: 1.0116594	test: 1.0090128	best: 1.0090128 (0)	total: 15.4ms	remaining: 12.8s
100:	learn: 0.0694137	test: 0.5038860	best: 0.5006817 (76)	total: 872ms	remaining: 6.32s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.4993713684
bestIteration = 106

Shrink model to first 107 iterations.


In [34]:
metric(y_val, model.predict(x_val))

0.8510703912442845

# SGD

In [35]:
from sklearn.linear_model import SGDClassifier

In [42]:
def objective(trial):
    params = {
        'loss': 'log_loss',
        'penalty': 'l2',
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 2000),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'optimal', 'invscaling', 'adaptive']),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'eta0': trial.suggest_float('eta0', 1e-8, 10.0, log=True),
        'early_stopping': True,
        'random_state': 42
    }
    
    model = SGDClassifier(**params)
    model.fit(x_train, y_train)
    
    preds = model.predict(x_val)
    
    return metric(y_val, preds)

In [43]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-09-04 09:34:21,250] A new study created in memory with name: no-name-c63ff930-f5d6-4df9-b954-e6bc895cd8c7
[I 2024-09-04 09:34:21,281] Trial 0 finished with value: 0.6344234079173838 and parameters: {'alpha': 0.00022832222963530113, 'max_iter': 1559, 'learning_rate': 'optimal', 'class_weight': None, 'eta0': 0.00021084023337233581}. Best is trial 0 with value: 0.6344234079173838.
[I 2024-09-04 09:34:21,396] Trial 1 finished with value: 0.4625322162635596 and parameters: {'alpha': 4.715661429581094, 'max_iter': 649, 'learning_rate': 'adaptive', 'class_weight': None, 'eta0': 0.7409506388681651}. Best is trial 0 with value: 0.6344234079173838.
[I 2024-09-04 09:34:21,421] Trial 2 finished with value: 0.8079570699959323 and parameters: {'alpha': 0.025404330456478122, 'max_iter': 1497, 'learning_rate': 'constant', 'class_weight': 'balanced', 'eta0': 0.004597468184550306}. Best is trial 2 with value: 0.8079570699959323.
[I 2024-09-04 09:34:21,452] Trial 3 finished with value: 0.76564828

[I 2024-09-04 09:34:23,137] Trial 30 finished with value: 0.6813492063492063 and parameters: {'alpha': 0.007673091736983274, 'max_iter': 502, 'learning_rate': 'constant', 'class_weight': None, 'eta0': 0.2977509158909941}. Best is trial 2 with value: 0.8079570699959323.
[I 2024-09-04 09:34:23,197] Trial 31 finished with value: 0.7868686868686869 and parameters: {'alpha': 2.1836063824005125e-06, 'max_iter': 1254, 'learning_rate': 'constant', 'class_weight': None, 'eta0': 0.002828831986119102}. Best is trial 2 with value: 0.8079570699959323.
[I 2024-09-04 09:34:23,264] Trial 32 finished with value: 0.7548500881834216 and parameters: {'alpha': 1.4250297347311692e-07, 'max_iter': 1419, 'learning_rate': 'constant', 'class_weight': None, 'eta0': 0.00044831618706430354}. Best is trial 2 with value: 0.8079570699959323.
[I 2024-09-04 09:34:23,344] Trial 33 finished with value: 0.7656641604010025 and parameters: {'alpha': 7.470954548475915e-06, 'max_iter': 1023, 'learning_rate': 'constant', 'clas

In [44]:
study.best_params

{'alpha': 0.025404330456478122,
 'max_iter': 1497,
 'learning_rate': 'constant',
 'class_weight': 'balanced',
 'eta0': 0.004597468184550306}

In [45]:
model = SGDClassifier(**study.best_params, loss='log_loss', penalty='l2', early_stopping=True, random_state=42)
model.fit(x_train, y_train)

SGDClassifier(alpha=0.025404330456478122, class_weight='balanced',
              early_stopping=True, eta0=0.004597468184550306,
              learning_rate='constant', loss='log_loss', max_iter=1497,
              random_state=42)

In [46]:
metric(y_val, model.predict(x_val))

0.8079570699959323

# XGBoost

In [47]:
from xgboost import XGBClassifier
from sklearn.utils import compute_sample_weight

In [48]:
balanced_weights = compute_sample_weight(class_weight='balanced', y=y_train)

In [49]:
def objective(trial):
    params = {
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'sample_weight': trial.suggest_categorical('sample_weight', ['balanced', None]),
        'early_stopping_rounds': 30,
        'verbosity': 0
    }
    
    if params['sample_weight'] == 'balanced':
        params['sample_weight'] = balanced_weights
        
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
    
    preds = model.predict(x_val)
    
    return metric(y_val, preds)

In [50]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-09-04 09:44:18,937] A new study created in memory with name: no-name-48379cad-f1c7-4da1-b83a-7e14e3e86b9b
[I 2024-09-04 09:44:20,012] Trial 0 finished with value: 0.7966751918158569 and parameters: {'reg_alpha': 0.2531289614160166, 'reg_lambda': 0.010169999610824767, 'num_leaves': 93, 'max_depth': 7, 'n_estimators': 734, 'learning_rate': 0.2165510412695304, 'colsample_bytree': 0.44352875188151397, 'subsample': 0.4483604928565383, 'booster': 'dart', 'sample_weight': None}. Best is trial 0 with value: 0.7966751918158569.
[I 2024-09-04 09:44:20,193] Trial 1 finished with value: 0.7859177859177859 and parameters: {'reg_alpha': 2.821094501469935e-08, 'reg_lambda': 0.00011185642242501595, 'num_leaves': 219, 'max_depth': 2, 'n_estimators': 399, 'learning_rate': 0.26827883571002775, 'colsample_bytree': 0.896436711847661, 'subsample': 0.4526667854457149, 'booster': 'gbtree', 'sample_weight': None}. Best is trial 0 with value: 0.7966751918158569.
[I 2024-09-04 09:44:22,842] Trial 2 finis

[I 2024-09-04 09:47:43,678] Trial 19 finished with value: 0.8183417999968349 and parameters: {'reg_alpha': 1.19167699368429e-08, 'reg_lambda': 1.4377050082848331e-05, 'num_leaves': 122, 'max_depth': 3, 'n_estimators': 482, 'learning_rate': 0.06292790425869609, 'colsample_bytree': 0.682997960096475, 'subsample': 0.710772954632176, 'booster': 'dart', 'sample_weight': 'balanced'}. Best is trial 16 with value: 0.8517625629486768.
[I 2024-09-04 09:47:46,161] Trial 20 finished with value: 0.7966070202621419 and parameters: {'reg_alpha': 0.00015187202793707081, 'reg_lambda': 0.005388148956866303, 'num_leaves': 229, 'max_depth': 7, 'n_estimators': 711, 'learning_rate': 0.15715363439416186, 'colsample_bytree': 0.4817808445836964, 'subsample': 0.34415274312794303, 'booster': 'dart', 'sample_weight': 'balanced'}. Best is trial 16 with value: 0.8517625629486768.
[I 2024-09-04 09:47:54,841] Trial 21 finished with value: 0.8183417999968349 and parameters: {'reg_alpha': 0.034813218596817935, 'reg_lam

[I 2024-09-04 09:48:12,095] Trial 38 finished with value: 0.8161904761904761 and parameters: {'reg_alpha': 2.887395276624162e-07, 'reg_lambda': 0.00032076645055382466, 'num_leaves': 237, 'max_depth': 3, 'n_estimators': 1065, 'learning_rate': 0.07575153354428628, 'colsample_bytree': 0.734917868054002, 'subsample': 0.7363073509521034, 'booster': 'gbtree', 'sample_weight': None}. Best is trial 16 with value: 0.8517625629486768.
[I 2024-09-04 09:48:12,699] Trial 39 finished with value: 0.7947144075021312 and parameters: {'reg_alpha': 0.027935387156988754, 'reg_lambda': 2.4939447365960207e-06, 'num_leaves': 183, 'max_depth': 6, 'n_estimators': 1199, 'learning_rate': 0.14353159696826412, 'colsample_bytree': 0.4680280815123454, 'subsample': 0.5188080721837331, 'booster': 'dart', 'sample_weight': 'balanced'}. Best is trial 16 with value: 0.8517625629486768.
[I 2024-09-04 09:48:12,828] Trial 40 finished with value: 0.7870133560670646 and parameters: {'reg_alpha': 6.884857648139545e-05, 'reg_lam

In [51]:
study.best_params

{'reg_alpha': 5.343267100695218e-05,
 'reg_lambda': 0.0001822072939479471,
 'num_leaves': 182,
 'max_depth': 5,
 'n_estimators': 627,
 'learning_rate': 0.06972132916589005,
 'colsample_bytree': 0.8605585856220815,
 'subsample': 0.792730808376586,
 'booster': 'dart',
 'sample_weight': 'balanced'}

In [55]:
params = study.best_params
params['sample_weight'] = balanced_weights
params['verbosity'] = 0
params['early_stopping_rounds'] = 30

model = XGBClassifier(**params)
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8605585856220815, device=None,
              early_stopping_rounds=30, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06972132916589005,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=627, n_jobs=None,
              num_leaves=182, num_parallel_tree=None, ...)

In [56]:
metric(y_val, model.predict(x_val))

0.8517625629486768